In [349]:
"""
Скрипт для склонения ФИО по падежам
"""

'\nСкрипт для склонения ФИО по падежам\n'

In [350]:
import pandas as pd
import pytrovich
from pytrovich.detector import PetrovichGenderDetector
from pytrovich.enums import NamePart, Gender, Case
from pytrovich.maker import PetrovichDeclinationMaker
import time



In [351]:
def capitalize_double_name(word):
    """
    Функция для того чтобы в двойных именах и фамилиях вторая часть была также с большой буквы
    """
    lst_word = word.split('-') # сплитим по дефису
    if len(lst_word) == 1: # если длина списка равна 1 то это не двойная фамилия и просто возвращаем слово
        
        return word
    elif len(lst_word) == 2:
        first_word = lst_word[0].capitalize() # делаем первую букву слова заглавной а остальные строчными
        second_word = lst_word[1].capitalize()
        return f'{first_word}-{second_word}'
    else:
        return 'Не удалось просклонять'

def case_lastname(maker,lastname,gender,case:Case):
    """
    Функция для обработки и склонения фамилии. Это нужно для обработки случаев двойной фамилии
    """
    
    lst_lastname = lastname.split('-') # сплитим по дефису
    
    if len(lst_lastname) == 1: # если длина списка равна 1 то это не двойная фамилия и просто обрабатываем слово
        case_result_lastname = maker.make(NamePart.LASTNAME,gender,case,lastname)
        return case_result_lastname
    elif len(lst_lastname) == 2:       
        first_lastname = lst_lastname[0].capitalize() # делаем первую букву слова заглавной а остальные строчными
        second_lastname = lst_lastname[1].capitalize()
        # Склоняем по отдельности
        first_lastname = maker.make(NamePart.LASTNAME,gender,case,first_lastname)
        second_lastname = maker.make(NamePart.LASTNAME,gender,case,second_lastname)        
        
        return f'{first_lastname}-{second_lastname}'
    
           


def detect_gender(detector,lastname,firstname,middlename):
    """
    Функция для определения гендера слова
    """
#     detector = PetrovichGenderDetector() # создаем объект детектора
    try:
        gender_result = detector.detect(lastname=lastname,firstname=firstname,middlename=middlename)
        return gender_result
    except StopIteration:# если не удалось определить то считаем что гендер андрогинный
        return Gender.ANDROGYNOUS
       
    

def decl_on_case(fio:str,case:Case) ->str:
    """
    Функция для склонения ФИО по падежам
    """
    fio = fio.strip() # очищаем строку от пробельных символов с начала и конца
    part_fio = fio.split() # разбиваем по пробелам создавая список где [0] это Фамилия,[1]-Имя,[2]-Отчество
    
    if len(part_fio) == 3: # проверяем на длину и обрабатываем только те что имеют длину 3 во всех остальных случаях просим просклонять самостоятельно
        maker = PetrovichDeclinationMaker() # создаем объект класса
        lastname = part_fio[0].capitalize() # Фамилия
        firstname = part_fio[1].capitalize() # Имя
        middlename = part_fio[2].capitalize() # Отчество
        
        
        # Определяем гендер для корректного склонения
        detector = PetrovichGenderDetector() # создаем объект детектора
        gender = detect_gender(detector,lastname,firstname,middlename)
        # Склоняем
        
        
        case_result_lastname= case_lastname(maker,lastname,gender,case) # обрабатываем фамилию
#         case_result_lastname = maker.make(NamePart.LASTNAME,gender,case,lastname)
#         case_result_lastname = capitalize_double_name(case_result_lastname) # обрабатываем случаи двойной фамилии
        case_result_firstname = maker.make(NamePart.FIRSTNAME,gender,case,firstname)
        case_result_firstname= capitalize_double_name(case_result_firstname)# обрабатываем случаи двойного имени
        
        case_result_middlename = maker.make(NamePart.MIDDLENAME,gender,case,middlename)
        # Возвращаем результат
        result_fio = f'{case_result_lastname} {case_result_firstname} {case_result_middlename}'
        return result_fio
        
        
        
    else:
        return 'Проверьте количество слов, должно быть 3 разделенных пробелами слова'

In [352]:
fio_column = 'ФИО'
data_decl_case = 'data/краткий вариант 37 строк.xlsx'

In [353]:
# df = pd.read_excel('data/cases.xlsx',dtype={fio_column:str})
df = pd.read_excel(data_decl_case,dtype={fio_column:str})

temp_df = pd.DataFrame() # временный датафрейм для хранения колонок просклоненных по падежам
path_to_end_folder_decl_case = 'data'

In [354]:
# Получаем номер колонки с фио которые нужно обработать
lst_columns = list(df.columns) # Превращаем в список
index_fio_column = lst_columns.index(fio_column) # получаем индекс




In [355]:
# Обрабатываем nan значения и те которые обозначены пробелом
df[fio_column].fillna('Не заполнено',inplace=True)
df[fio_column] = df[fio_column].apply(lambda x:x.strip())
df[fio_column] = df[fio_column].apply(lambda x:x if x else 'Не заполнено') # Если пустая строка то заменяем на значение Не заполнено

In [356]:
temp_df['Родительный_падеж'] = df[fio_column].apply(lambda x:decl_on_case(x,Case.GENITIVE))
temp_df['Дательный_падеж'] = df[fio_column].apply(lambda x:decl_on_case(x,Case.DATIVE))
temp_df['Винительный_падеж'] = df[fio_column].apply(lambda x:decl_on_case(x,Case.ACCUSATIVE))
temp_df['Творительный_падеж'] = df[fio_column].apply(lambda x:decl_on_case(x,Case.INSTRUMENTAL))
temp_df['Предложный_падеж'] = df[fio_column].apply(lambda x:decl_on_case(x,Case.PREPOSITIONAL))

In [357]:
# Вставляем получившиеся колонки после базовой колонки с фио
df.insert(index_fio_column+1,'Родительный_падеж',temp_df['Родительный_падеж'])
df.insert(index_fio_column+2,'Дательный_падеж',temp_df['Дательный_падеж'])
df.insert(index_fio_column+3,'Винительный_падеж',temp_df['Винительный_падеж'])
df.insert(index_fio_column+4,'Творительный_падеж',temp_df['Творительный_падеж'])
df.insert(index_fio_column+5,'Предложный_падеж',temp_df['Предложный_падеж'])

In [358]:
t = time.localtime()
current_time = time.strftime('%H_%M_%S', t)
df.to_excel(f'{path_to_end_folder_decl_case}/ФИО по падежам от {current_time}.xlsx',index=False)